survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [3]:
import pandas as pd
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [364]:
raw = pd.read_csv('train.csv', index_col='PassengerId')
test = pd.read_csv('test.csv', index_col='PassengerId')
raw.columns = pd.Series(raw.columns).apply(lambda r: r.lower())
raw.sex = raw.sex.map({'female': 0, 'male': 1})
raw.head(2)

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C


In [365]:
test.head(2)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [366]:
L=[900,1,1,'lenka',0,29,0,0,'rrr',70,'tt','S']
pd.DataFrame(L).T

,0,1,2,3,4,5,6,7,8,9,10,11
0,900,1,1,lenka,0,29,0,0,rrr,70,tt,S


In [367]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
survived    891 non-null int64
pclass      891 non-null int64
name        891 non-null object
sex         891 non-null int64
age         714 non-null float64
sibsp       891 non-null int64
parch       891 non-null int64
ticket      891 non-null object
fare        891 non-null float64
cabin       204 non-null object
embarked    889 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 83.5+ KB


In [368]:
#raw.pivot_table(index=['survived','sex','embarked'],columns=['pclass'],values='name',aggfunc='count')
raw.age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [332]:
f1=raw[raw.survived==1].groupby(['pclass','age']).name.count()
f2=raw[raw.survived==0].groupby(['age']).name.count()
#pd.DataFrame(f1)
r=raw.pivot_table(index=['sex','pclass'],columns = ['survived'],values='age',aggfunc='mean')
r
#plt.bar(f2.index,f2,color='red')
#plt.bar(f1.index,f1,color='green')


survived            0          1
sex pclass                      
0   1       25.666667  34.939024
    2       36.000000  28.080882
    3       23.818182  19.329787
1   1       44.581967  36.248000
    2       33.369048  16.022000
    3       27.255814  22.274211

In [333]:
r[0][0]

pclass
1    25.666667
2    36.000000
3    23.818182
Name: 0, dtype: float64

In [363]:
#raw.age=raw.age.fillna(-1)
#tmp=pd.Series()
#raw[raw.age==-1]
#for p in raw.loc[raw.age==-1].index:
#    print p,
#    print type(p)
#    tmp.append(raw[p][p.survived][p.sex][p.pclass])

In [369]:
raw.age=raw.age.fillna(38)
raw.cabin = raw.cabin.fillna('na') # не будем использовать
raw.embarked= raw.embarked.fillna('S') # либо С, т.к. это только пассажиры первого класса

In [370]:
print raw[raw.name.str.contains('Miss')].age.mean()
print raw[raw.name.str.contains('Mrs.')].age.mean()
print raw[raw.name.str.contains('Mr.')].age.mean()

24.9835164835
35.9534883721
34.1445131376


In [123]:
raw.columns

Index([u'passengerid', u'survived', u'pclass', u'name', u'sex', u'age',
       u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked'],
      dtype='object')

In [371]:
num_data = raw[['pclass','sex', 'age','sibsp', 'parch','fare']]
cat_data = raw[['cabin','embarked']].drop(['cabin'],axis=1)
y=raw['survived']
cat_data.head(2)

,embarked
PassengerId,
1,S
2,C


In [372]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [424]:
scaler = StandardScaler()
transform = PolynomialFeatures(1)


X_s=scaler.fit_transform(num_data)
X_cat= encoder.fit_transform(cat_data.T.to_dict().values())
X_s=np.hstack([X_s,X_cat])

X_s = transform.fit_transform(X_s)
#pd.DataFrame(X)

encoder = DV(sparse = False)
X=X_s
pd.DataFrame(X).head(2)

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.827377,0.737695,-0.697093,0.432793,-0.473674,-0.502445,0.0,0.0,1.0
1,1.0,-1.566107,-1.355574,0.496034,0.432793,-0.473674,0.786845,1.0,0.0,0.0


In [444]:
lg=pd.Series([0])
xg=[0]
for i in range(0,16):    
    (X_train,
    X_test,
    y_train,
    y_test)=train_test_split(X,y,test_size=0.3,stratify=y)

    xgb = XGBClassifier()
    xgb.fit(X_train,y_train)

    LG = LogisticRegression(class_weight='balanced',penalty='l1')
    LG.fit(X_train,y_train)
    
    lg=lg.append(roc_auc_score(y_test,LG.predict(X_test)))
    xg=xg.append(roc_auc_score(y_test,xgb.predict(X_test)))


print ' LG : {} '.format(lg.mean())#roc_auc_score(y_test,LG.predict(X_test)))
print ' XGB : {} '.format(xg.mean())#roc_auc_score(y_test,xgb.predict(X_test)))

TypeError: cannot concatenate a non-NDFrame object

In [228]:
print raw.columns
pd.DataFrame(LG.coef_)


Index([u'passengerid', u'survived', u'pclass', u'name', u'sex', u'age',
       u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked'],
      dtype='object')


,0,1,2,3,4,5,6,7,8,9
0,-0.856971,-1.177105,-0.543908,-0.324722,-0.04881,0.160789,0.183599,0.0,-0.318912,0.0
